#### this notebook sends requests using restaurants names and Places API to collect place info

In [1]:
import pandas as pd
import requests

In [2]:
datafile = 'data/restaurant_list_Toronto_tripAdvisor.csv'
res_df   = pd.read_csv(datafile)

In [3]:
res_df['name2'] = res_df['name'].apply(lambda name: name[name.find('.')+2:])

In [4]:
print(len(res_df[res_df.duplicated(subset=['name2'])]))
res_df2 = res_df.drop_duplicates(subset=['name2'])
print(len(res_df))
print(len(res_df2))

115
2100
1985


In [5]:
# rectangle_northWest = ('43.791765', '-79.156357')
# rectangle_northWest

In [5]:
locationbiasLat = 43.7064516
locationbiasLng = -79.399816
radius = 20000 #20km
rect_northEast_lat = '43.791765'
rect_northEast_lng = '-79.156357'
rect_southWest_lat = '43.569896', 
rect_southWest_lng = '-79.551885'
myKey = ''
def get_place_info (name):
    search_word = name.replace(' ','%20') #space in html
    textQuery = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={search_word}&inputtype=textquery&fields=name,rating,formatted_address,place_id,business_status,geometry&locationbias=rectangle:{rect_southWest_lat},{rect_southWest_lng}|{rect_northEast_lat},{rect_northEast_lng}&key=myKey"
    response = requests.get(textQuery)
    return response.json()['candidates']

res_df3 = res_df2.copy()
res_df3['json'] = res_df2['name2'].apply(lambda name: get_place_info(name))


In [8]:
#drop those with no info retrived
info = res_df3['json'].apply(lambda x: len(x))
info[info==0] # four restaurants with no info retrieved
res_df3.drop(res_df3[info==0].index,inplace=True)

In [9]:
#check that it worked
info = res_df3['json'].apply(lambda x: len(x))
(info==0).any()

False

In [10]:
# how many have more than 1 candidate
print(len(info[info>1]))
# how many have more than 2 candidates
print(len(info[info>2]))

193
0


In [160]:
# res_df3[info>1]

In [11]:
# if we restrict to Toronto addresses only
onlyTo = res_df3[res_df3['json'].apply(lambda x:'Toronto' in x[0]['formatted_address'])].copy()
len(onlyTo)

1493

In [12]:
# There are still 132 with double address
info = onlyTo['json'].apply(lambda x:len(x))
print(len(info[info>1]))
del onlyTo

134


In [15]:
#unwrap the info and add each as a column
res_df3['name_google_0']        = res_df3['json'].apply(lambda json:json[0]['name'])
res_df3['place_id_0']           = res_df3['json'].apply(lambda json:json[0]['place_id'])
res_df3['formatted_address_0']  = res_df3['json'].apply(lambda json:json[0]['formatted_address'])
res_df3['business_status_0']    = res_df3['json'].apply(lambda json:json[0]['business_status'] if ('business_status') in json[0] else None)
res_df3['lat_0']                = res_df3['json'].apply(lambda json:json[0]['geometry']['location']['lat'])
res_df3['lng_0']                = res_df3['json'].apply(lambda json:json[0]['geometry']['location']['lng'])
res_df3['rating_google_0']     = res_df3['json'].apply(lambda json:json[0]['rating'] if ('rating') in json[0] else None)


In [16]:
# Add second candiate info if there is one

res_df3['name_google_1']        = res_df3['json'].apply(lambda json:json[1]['name'] if len(json)>1 else None)
res_df3['place_id_1']           = res_df3['json'].apply(lambda json:json[1]['place_id'] if len(json)>1 else None)
res_df3['formatted_address_1']  = res_df3['json'].apply(lambda json:json[1]['formatted_address'] if len(json)>1 else None)
res_df3['business_status_1']    = res_df3['json'].apply(lambda json:json[1]['business_status'] if (len(json)>1 and 'business_status' in json[1]) else None)
res_df3['lat_1']                = res_df3['json'].apply(lambda json:json[1]['geometry']['location']['lat'] if len(json)>1 else None)
res_df3['lng_1']                = res_df3['json'].apply(lambda json:json[1]['geometry']['location']['lng'] if len(json)>1 else None)
res_df3['rating_google_1']      = res_df3['json'].apply(lambda json:json[1]['rating'] if len(json)>1 else None)


In [18]:
res_df3

,Unnamed: 0,name,stars,cuisine,retrieve-date,name2,json,name_google_0,place_id_0,formatted_address_0,...,lat_0,lng_0,rating_google_0,name_google_1,place_id_1,formatted_address_1,business_status_1,lat_1,lng_1,rating_google_1
0,0,1. Goodfellas Wood Oven Pizza,4.5 of 5 bubbles,"Italian, Pizza",2021-08-09 19:15:40.484368-04:00,Goodfellas Wood Oven Pizza,"[{'business_status': 'OPERATIONAL', 'formatted...",Goodfellas Wood Oven Pizza,ChIJedJ0GGI2K4gR6dco_CjRntY,"1 Old Mill Dr, York, ON M6S 0A1, Canada",...,43.648221,-79.486544,4.3,None,None,None,None,NaN,NaN,NaN
1,1,2. Moxie's Grill & Bar,4.5 of 5 bubbles,"Grill, Bar",2021-08-09 19:15:40.732702-04:00,Moxie's Grill & Bar,"[{'business_status': 'OPERATIONAL', 'formatted...",Moxie's Grill & Bar,ChIJfQJkpzMyK4gRDnUt3YFXa5w,"3401 Dufferin St #30, Toronto, ON M6A 2T9, Canada",...,43.725108,-79.452809,4.0,None,None,None,None,NaN,NaN,NaN
2,2,3. New Orleans Seafood & Steakhouse,4.5 of 5 bubbles,"Steakhouse, Cajun & Creole",2021-08-09 19:15:40.732702-04:00,New Orleans Seafood & Steakhouse,"[{'business_status': 'OPERATIONAL', 'formatted...",New Orleans Seafood & Steakhouse,ChIJ466I-9o2K4gRxzeisSIQUzg,"267 Scarlett Rd, York, ON M6N 4K9, Canada",...,43.677756,-79.506246,4.8,None,None,None,None,NaN,NaN,NaN
3,3,4. Scaramouche Restaurant,4.5 of 5 bubbles,"French, International",2021-08-09 19:15:40.732702-04:00,Scaramouche Restaurant,"[{'business_status': 'OPERATIONAL', 'formatted...",Scaramouche Restaurant,ChIJrxCATO8zK4gRHe57JQWul2Q,"1 Benvenuto Pl, Toronto, ON M4V 2L1, Canada",...,43.681502,-79.399901,4.7,None,None,None,None,NaN,NaN,NaN
4,4,5. ALO RESTAURANT,4.5 of 5 bubbles,"French, European",2021-08-09 19:15:40.732702-04:00,ALO RESTAURANT,"[{'business_status': 'OPERATIONAL', 'formatted...",Alo,ChIJryijc9s0K4gRG9aU7SDTXdA,"163 Spadina Ave., Toronto, ON M5V 2L6, Canada",...,43.648491,-79.395941,4.7,None,None,None,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,2095,2096. The Burren Pub,4.5 of 5 bubbles,Pub,2021-08-09 19:23:06.617804-04:00,The Burren Pub,"[{'business_status': 'OPERATIONAL', 'formatted...",The Burren Pub,ChIJeT8ioLPL1IkRNS2ses5dOUY,"1301 Queen St E, Toronto, ON M4L 1C2, Canada",...,43.663504,-79.328004,4.5,None,None,None,None,NaN,NaN,NaN
2096,2096,2097. The Beaver Cafe,4.0 of 5 bubbles,"Bar, Cafe",2021-08-09 19:23:06.617804-04:00,The Beaver Cafe,"[{'business_status': 'CLOSED_PERMANENTLY', 'fo...",Beaver Cafe,ChIJBx_Gvao1K4gRrFGEV1E0Ejs,"1192 Queen St W, Toronto, ON M6J 1J6, Canada",...,43.642778,-79.426389,4.5,None,None,None,None,NaN,NaN,NaN
2097,2097,2098. Tot Cat Cafe,3.5 of 5 bubbles,Cafe,2021-08-09 19:23:06.617804-04:00,Tot Cat Cafe,"[{'business_status': 'CLOSED_PERMANENTLY', 'fo...",Tot The Cat Cafe,ChIJAc4ay8E0K4gRlFr93DwkSdY,"298 College St, Toronto, ON M5T 1R9, Canada",...,43.657842,-79.401591,3.8,None,None,None,None,NaN,NaN,NaN
2098,2098,2099. Gatto Wood Oven Pizza,4.0 of 5 bubbles,"Italian, Pizza",2021-08-09 19:23:06.617804-04:00,Gatto Wood Oven Pizza,"[{'business_status': 'OPERATIONAL', 'formatted...",GATTO Wood Oven Pizza,ChIJReU0Gyo3K4gRh1aes3ounbE,"2312 Bloor St W, Toronto, ON M6S 1P2, Canada",...,43.650587,-79.479779,4.4,None,None,None,None,NaN,NaN,NaN


In [ ]:
for 

In [138]:
# build the googlempas URL and add it to the dataFrame
def get_google_maps_url(row):
    locationName = row[]


[{'business_status': 'OPERATIONAL',
  'formatted_address': '1 Old Mill Dr, York, ON M6S 0A1, Canada',
  'geometry': {'location': {'lat': 43.6482211, 'lng': -79.4865442},
   'viewport': {'northeast': {'lat': 43.64970152989272,
     'lng': -79.48478742010727},
    'southwest': {'lat': 43.64700187010727, 'lng': -79.48748707989272}}},
  'name': 'Goodfellas Wood Oven Pizza',
  'place_id': 'ChIJedJ0GGI2K4gR6dco_CjRntY',
  'rating': 4.3}]

In [19]:
res_df4 = res_df3.copy()
temp0 = []
temp1 = []
for ind,value in res_df3.iterrows():
    locationName = value['name_google_0']
    locationlat  = value['lat_0']
    locationlng  = value['lng_0']
    place_id     = value['place_id_0']
    temp0.append(f'https://www.google.com/maps/search/?api=1&query={locationName},{locationlat}%2C{locationlng}&query_place_id={place_id}')
    
    locationName = value['name_google_1']
    locationlat  = value['lat_1']
    locationlng  = value['lng_1']
    place_id     = value['place_id_1']
    if locationName != None:
        temp1.append(f'https://www.google.com/maps/search/?api=1&query={locationName},{locationlat}%2C{locationlng}&query_place_id={place_id}')
    else:
        temp1.append(None)

In [20]:
res_df4['url_0'] = temp0
res_df4['url_1'] = temp1

In [21]:
res_df4

,Unnamed: 0,name,stars,cuisine,retrieve-date,name2,json,name_google_0,place_id_0,formatted_address_0,...,rating_google_0,name_google_1,place_id_1,formatted_address_1,business_status_1,lat_1,lng_1,rating_google_1,url_0,url_1
0,0,1. Goodfellas Wood Oven Pizza,4.5 of 5 bubbles,"Italian, Pizza",2021-08-09 19:15:40.484368-04:00,Goodfellas Wood Oven Pizza,"[{'business_status': 'OPERATIONAL', 'formatted...",Goodfellas Wood Oven Pizza,ChIJedJ0GGI2K4gR6dco_CjRntY,"1 Old Mill Dr, York, ON M6S 0A1, Canada",...,4.3,None,None,None,None,NaN,NaN,NaN,https://www.google.com/maps/search/?api=1&quer...,None
1,1,2. Moxie's Grill & Bar,4.5 of 5 bubbles,"Grill, Bar",2021-08-09 19:15:40.732702-04:00,Moxie's Grill & Bar,"[{'business_status': 'OPERATIONAL', 'formatted...",Moxie's Grill & Bar,ChIJfQJkpzMyK4gRDnUt3YFXa5w,"3401 Dufferin St #30, Toronto, ON M6A 2T9, Canada",...,4.0,None,None,None,None,NaN,NaN,NaN,https://www.google.com/maps/search/?api=1&quer...,None
2,2,3. New Orleans Seafood & Steakhouse,4.5 of 5 bubbles,"Steakhouse, Cajun & Creole",2021-08-09 19:15:40.732702-04:00,New Orleans Seafood & Steakhouse,"[{'business_status': 'OPERATIONAL', 'formatted...",New Orleans Seafood & Steakhouse,ChIJ466I-9o2K4gRxzeisSIQUzg,"267 Scarlett Rd, York, ON M6N 4K9, Canada",...,4.8,None,None,None,None,NaN,NaN,NaN,https://www.google.com/maps/search/?api=1&quer...,None
3,3,4. Scaramouche Restaurant,4.5 of 5 bubbles,"French, International",2021-08-09 19:15:40.732702-04:00,Scaramouche Restaurant,"[{'business_status': 'OPERATIONAL', 'formatted...",Scaramouche Restaurant,ChIJrxCATO8zK4gRHe57JQWul2Q,"1 Benvenuto Pl, Toronto, ON M4V 2L1, Canada",...,4.7,None,None,None,None,NaN,NaN,NaN,https://www.google.com/maps/search/?api=1&quer...,None
4,4,5. ALO RESTAURANT,4.5 of 5 bubbles,"French, European",2021-08-09 19:15:40.732702-04:00,ALO RESTAURANT,"[{'business_status': 'OPERATIONAL', 'formatted...",Alo,ChIJryijc9s0K4gRG9aU7SDTXdA,"163 Spadina Ave., Toronto, ON M5V 2L6, Canada",...,4.7,None,None,None,None,NaN,NaN,NaN,https://www.google.com/maps/search/?api=1&quer...,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,2095,2096. The Burren Pub,4.5 of 5 bubbles,Pub,2021-08-09 19:23:06.617804-04:00,The Burren Pub,"[{'business_status': 'OPERATIONAL', 'formatted...",The Burren Pub,ChIJeT8ioLPL1IkRNS2ses5dOUY,"1301 Queen St E, Toronto, ON M4L 1C2, Canada",...,4.5,None,None,None,None,NaN,NaN,NaN,https://www.google.com/maps/search/?api=1&quer...,None
2096,2096,2097. The Beaver Cafe,4.0 of 5 bubbles,"Bar, Cafe",2021-08-09 19:23:06.617804-04:00,The Beaver Cafe,"[{'business_status': 'CLOSED_PERMANENTLY', 'fo...",Beaver Cafe,ChIJBx_Gvao1K4gRrFGEV1E0Ejs,"1192 Queen St W, Toronto, ON M6J 1J6, Canada",...,4.5,None,None,None,None,NaN,NaN,NaN,https://www.google.com/maps/search/?api=1&quer...,None
2097,2097,2098. Tot Cat Cafe,3.5 of 5 bubbles,Cafe,2021-08-09 19:23:06.617804-04:00,Tot Cat Cafe,"[{'business_status': 'CLOSED_PERMANENTLY', 'fo...",Tot The Cat Cafe,ChIJAc4ay8E0K4gRlFr93DwkSdY,"298 College St, Toronto, ON M5T 1R9, Canada",...,3.8,None,None,None,None,NaN,NaN,NaN,https://www.google.com/maps/search/?api=1&quer...,None
2098,2098,2099. Gatto Wood Oven Pizza,4.0 of 5 bubbles,"Italian, Pizza",2021-08-09 19:23:06.617804-04:00,Gatto Wood Oven Pizza,"[{'business_status': 'OPERATIONAL', 'formatted...",GATTO Wood Oven Pizza,ChIJReU0Gyo3K4gRh1aes3ounbE,"2312 Bloor St W, Toronto, ON M6S 1P2, Canada",...,4.4,None,None,None,None,NaN,NaN,NaN,https://www.google.com/maps/search/?api=1&quer...,None


In [22]:
csvfile = 'data/restaurant_info_Toronto_tripAdvisor_v2.csv'
res_df4.to_csv(csvfile)

In [161]:
set(res_df3['business_status_1'])

{'CLOSED_PERMANENTLY', 'CLOSED_TEMPORARILY', None, 'OPERATIONAL'}